In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shakespeare_functions as sf
import shakespeare_dicts as sd

from importlib import reload

import re
import nltk
from nltk import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk import RegexpTokenizer

from autocorrect import spell

import spacy
import en_core_web_sm

In [2]:
line_df = pd.read_csv('../data/csv/ShakespeareCharacterLines_cleaned.csv', index_col = ['play', 'name', 'line_number'])

In [3]:
colon = slice(None)

- Coriolanus is called Martius for part of the play. Make them the same character.

In [4]:
coriolanus = line_df.loc[('coriolanus', ['MARTIUS', 'CORIOLANUS'], colon), :].copy()

In [5]:
martius_line_max = coriolanus.loc[(colon, 'MARTIUS', colon), :].index.get_level_values(2).max()
coriolanus_line_max = coriolanus.loc[(colon, 'CORIOLANUS', colon), :].index.get_level_values(2).max()

In [6]:
added_index = [('coriolanus', 'CORIOLANUS', x) for x in range(coriolanus_line_max, coriolanus_line_max + martius_line_max + 1)]
coriolanus = pd.concat([coriolanus, pd.DataFrame(columns = coriolanus.columns, index = added_index)])

In [7]:
coriolanus_new_lines = coriolanus.loc[(colon, 'CORIOLANUS', colon), :].shift(martius_line_max)
coriolanus_new_lines.dropna(inplace = True)

In [8]:
martius_lines = coriolanus.loc[(colon, 'MARTIUS', colon), :].copy()
coriolanus = pd.concat([martius_lines, coriolanus_new_lines])

In [9]:
coriolanus.index = coriolanus.index.map(lambda i: (i[0], 'CORIOLANUS', i[2]))

In [10]:
coriolanus

line  \
play       name       line_number                                                      
coriolanus CORIOLANUS 1            Thanks.--What's the matter, you dissentious ro...   
                      2                 That, rubbing the poor itch of your opinion,   
                      3                                       Make yourselves scabs?   
                      4            He that will give good words to thee will flatter   
                      5            Beneath abhorring. What would you have, you curs,   
...                                                                              ...   
                      980                      Fluttered your Volscians in Corioles,   
                      981                                     Alone I did it. "Boy"!   
                      982                                         O, that I had him,   
                      983                   With six Aufidiuses, or more, his tribe,   
                      984                                    To use my lawful sword.   

                                  play_line_number line_length  
play       name       line_number                               
coriolanus CORIOLANUS 1                        176          51  
                      2                        177          44  
                      3                        178          22  
                      4                        180          49  
                      5                        181          49  
...                                            ...         ...  
                      980                     4032          37  
                      981                     4033          22  
                      982                     4047          20  
                      983                     4048          40  
                      984                     4049          23  

[942 rows x 3 columns]

In [11]:
line_df.drop(line_df.loc[('coriolanus', ['MARTIUS', 'CORIOLANUS'], colon), :].index, inplace = True)

In [12]:
line_df = pd.concat([line_df, coriolanus])

In [13]:
line_df.sort_index(inplace = True)

- Identify all characters that died by consulting the main texts and outside summaries for verification.

In [14]:
deaths = {'antony-and-cleopatra': ['ENOBARBUS', 'EROS', 'ANTONY', 'IRAS', 'CHARMIAN', 'CLEOPATRA'],
          'coriolanus': ['CORIOLANUS'],
          'cymbeline': ['QUEEN', 'CLOTEN'],
          'hamlet': ['POLONIUS', 'ROSENCRANTZ', 'GUILDENSTERN', 'KING', 'QUEEN', 'LAERTES', 'OPHELIA', 'HAMLET', 'OSRIC'],
          'henry-iv-part-1': ['HOTSPUR', 'VERNON', 'WORCESTER', 'BLUNT'],
          'henry-iv-part-2': ['KING'],
          'henry-v': ['HOSTESS'],
          'henry-vi-part-1': ['MORTIMER', 'BEDFORD', 'TALBOT', 'JOHN TALBOT', 'GARGRAVE', 'SALISBURY'],
          'henry-vi-part-2': ['CARDINAL', 'GLOUCESTER', 'SUFFOLK', 'CADE', 'SOMERSET', 'CLIFFORD', 'STAFFORD', 'BROTHER', 'SAYE', 'CLERK', 'HORNER', 'SOLDIER'],
          'henry-vi-part-3': ['CLIFFORD', 'FATHER', 'SON', 'WARWICK', 'MONTAGUE', 'KING HENRY', 'RUTLAND', 'YORK', 'PRINCE EDWARD'],
          'henry-viii': ['WOLSEY', 'BUCKINGHAM'],
          'julius-caesar': ['TITINIUS', 'CASSIUS', 'CAESAR', 'CINNA', 'BRUTUS', 'CATO'],
          'king-john': ['AUSTRIA', 'ARTHUR', 'KING JOHN', 'CONSTANCE', 'QUEEN ELEANOR'],
          'king-lear': ['FIRST SERVANT', 'OSWALD', 'CORDELIA', 'REGAN', 'EDMUND', 'GLOUCESTER', 'LEAR', 'CORNWALL', 'GONERIL'],
          'loves-labors-lost': ['KING'],
          'macbeth': ['DUNCAN', 'BANQUO', 'LADY MACBETH', 'YOUNG SIWARD', 'MACBETH', 'LADY MACDUFF', 'SON'],
          'othello': ['RODERIGO', 'EMILIA', 'OTHELLO', 'DESDEMONA'], 
          'pericles': ['CLEON', 'DIONYZA', 'ANTIOCHUS', 'DAUGHTER'],
          'richard-ii': ['GAUNT', 'BUSHY', 'GREEN', 'RICHARD', 'GLOUCESTER'],
          'richard-iii': ['CLARENCE', 'RICHARD', 'KING EDWARD', 'PRINCE', 'YORK', 'RIVERS', 'GREY', 'VAUGHAN', 'BRAKENBURY', 'BUCKINGHAM', 'ANNE', 'HASTINGS'],
          'romeo-and-juliet': ['ROMEO', 'JULIET', 'MERCUTIO', 'TYBALT', 'PARIS', 'LADY MONTAGUE'],
          'the-two-noble-kinsmen': ['ARCITE'],
          'the-winters-tale': ['MAMILLIUS', 'HERMIONE', 'ANTIGONUS'], 
          'timon-of-athens': ['TIMON'], 
          'titus-andronicus': ['TITUS', 'MARTIUS', 'QUINTUS', 'MUTIUS', 'TAMORA', 'SATURNINUS', 'AARON', 'BASSIANUS', 'NURSE', 'LAVINIA', 'CHIRON', 'DEMETRIUS'],
          'troilus-and-cressida': ['HECTOR', 'PATROCLUS']}

- GLOUCESTER was originally DUCHESS in richard-ii, where there are two characters with line name DUCHESS: the duchess of York and the duchess of Gloucester. She has been renamed in the text itself.

In [15]:
line_df['character_dies'] = 0

for play, name_arr in deaths.items():
    for name in name_arr:
        line_df.loc[(play, name, colon), 'character_dies'] = 1

- Get number of words per line.

In [16]:
line_df['word_count'] = line_df['line']

In [17]:
line_df['word_count'] = line_df['word_count'].map(lambda line: len(re.findall(r'(\S+)(?=(\s*|\s*\Z))', line)))

- Add new features based on total character, word, and line count.

In [18]:
line_df['total_character_count'] = 0
line_df['max_line_count'] = 0

In [19]:
index_without_number = set(zip(line_df.index.get_level_values(0), line_df.index.get_level_values(1)))

In [20]:
for play, name in index_without_number:
    character_slice = line_df.loc[(play, name, colon)]
    
    line_df.loc[(play, name, colon), 'max_line_count'] = max(list(character_slice.index.get_level_values(2)))
    line_df.loc[(play, name, colon), 'total_character_count'] = sum(list(character_slice['line_length']))

- Add percent total character and line count features.

In [45]:
for play, name in index_without_number:
    character_slice = line_df.loc[(play, name, colon)]
    play_slice = line_df.loc[(play, colon, colon)]
    
    line_df.loc[(play, name, colon), 'percent_line_count'] = character_slice['max_line_count'] / max(list(play_slice['play_line_number']))
    line_df.loc[(play, name, colon), 'percent_word_count'] = sum(list(character_slice['word_count'])) / sum(list(play_slice['word_count']))

- Remove excess punctuation.

In [22]:
line_df['line'] = line_df['line'].map(sf.clean_punctuation)

- Separate all contractions, replace all poetic apostrophes with correct lettering.

In [23]:
line_df['line'] = line_df['line'].map(sf.clean_contractions)

- Remove "'s"s, they will be removed by lemmatizing or stopwords anyway.

In [24]:
line_df['line'] = line_df['line'].map(lambda x: re.sub("\'s", '', x))

- Modernize anachronistic/poetic words.

In [25]:
line_df['line'] = line_df['line'].map(sf.clean_anachronisms)

In [26]:
line_df['line'] = line_df['line'].map(lambda x: re.sub(r"-", ' ', x))

- Bring in spaCy for lemmatization, stopwords, NER, and POS tagging. 

In [27]:
nlp = en_core_web_sm.load()

- Add personalized stopwords.

In [28]:
for stop in sf.get_stopwords():
    nlp.vocab[stop].is_stop = True

- Get sentiment analysis of lines without proper nouns.

In [29]:
line_df.rename(columns = {'line': 'character_line'}, inplace = True)

In [30]:
line_df['pos_sentiment'] = 0
line_df['neg_sentiment'] = 0
line_df['neu_sentiment'] = 0
line_df['compound_sentiment'] = 0

In [31]:
sia = SentimentIntensityAnalyzer()

for play, name, line_number, character_line, play_line_number, line_length, character_dies, \
    word_count, total_character_count, max_line_count, \
    percent_line_count, percent_word_count, pos_sentiment, \
    neg_sentiment, neu_sentiment, compound_sentiment in line_df.to_records():
    
    line = character_line
    
    doc = nlp(line)
    to_sent = ""
    
    for token in doc:
        if token.pos_ != 'PROPN':
            to_sent += token.text + " "
       
    sent = sia.polarity_scores(to_sent)
    
    for key, value in sent.items():
        if line_df[f'{key}_sentiment'].dtypes != 'float32':
            line_df[f'{key}_sentiment'] = line_df[f'{key}_sentiment'].astype(dtype='float32')
        line_df.loc[(play, name, line_number), f'{key}_sentiment'] = value

In [32]:
line_df['compound_sentiment'] = line_df['compound_sentiment'].map(lambda x: (x + 1)/2)

- Final tokenization: remove stopwords, add columns for words, hypernyms, and word types

In [33]:
def dict_increment(a_dict, word):
    if word in a_dict:
        a_dict[word] += 1
    else:
        a_dict[word] = 1

In [34]:
%%time
character_names = set(line_df.index.get_level_values(1))

for play, name, line_number, character_line, play_line_number, line_length, character_dies, \
    word_count, total_character_count, max_line_count, \
    percent_line_count, percent_word_count, pos_sentiment, \
    neg_sentiment, neu_sentiment, compound_sentiment in line_df.to_records():
    
    line = character_line
    
    doc = nlp(line)
    block = {}
    
    for token in doc:
        features = []
        
        if token.text.upper() in character_names:
            features.extend(["character_name", "PROPN"])
        else:
            if not token.is_stop:
                features.extend([token.lemma_.lower(), token.pos_])

                syn_text = wn.synsets(token.text.lower())
                syn_lemma = wn.synsets(token.lemma_.lower())

                #Add hypernym/synonym columns in priority order: word hypernym > lemma hypernym > word synonym > lemma synonym.
                if len(syn_text) > 0 and len(syn_text[0].hypernyms()) > 0:
                    dict_increment(block, syn_text[0].hypernyms()[0].lemma_names()[0].lower() + "_hyp")
                elif len(syn_lemma) > 0 and len(syn_lemma[0].hypernyms()) > 0:
                    dict_increment(block, syn_lemma[0].hypernyms()[0].lemma_names()[0].lower() + "_hyp")
                elif len(syn_text) > 0:
                    dict_increment(block, syn_text[0].lemma_names()[0].lower() + "_syn")
                elif len(syn_lemma) > 0:
                    dict_increment(block, syn_lemma[0].lemma_names()[0].lower() + "_syn")
                
        for item in features:
            dict_increment(block, item)

    for word, count in block.items():
        if not word in line_df.columns:
            line_df[word] = 0
            line_df[word] = line_df[word].astype(dtype='uint16')
        line_df.loc[(play, name, line_number), word] = count

Wall time: 26min 16s


In [35]:
line_df.shape

(114295, 22147)

In [36]:
line_df.fillna(value = 0, inplace = True)

- Create aggregated dataframe by character for modeling.

In [37]:
to_drop = list(line_df.columns[:10])
to_drop.remove('character_dies')

In [38]:
char_df = line_df.drop(columns = to_drop).copy()

In [68]:
new_cols = [key + '_mean' for key in char_df.columns]
new_cols.extend([key + '_median' for key in char_df.columns])
new_cols.extend([key + '_std' for key in char_df.columns])

new_char_df = pd.DataFrame(index = index_without_number, columns = new_cols)

for play, name in index_without_number:
    character_slice = char_df.loc[(play, name, colon)]

    means = character_slice.mean().add_suffix('_mean')
    medians =   character_slice.median().add_suffix('_median')
    stds = character_slice.std(ddof = 0).add_suffix('_std')
    
    new_row = pd.concat([means, medians, stds], axis = 0)

    new_char_df.loc[(play, name)] = new_row
    

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line 248, in wrapped
    return f(*args, **kwargs)
  File "C:\ProgramData\Anaconda3\l

TypeError: object of type 'NoneType' has no len()

- Create History/Tragedy/Comedy frame for modeling.

In [51]:
comedies = ["alls-well-that-ends-well", "as-you-like-it", "the-comedy-of-errors", \
            "cymbeline", "loves-labors-lost", "measure-for-measure", \
            "the-merchant-of-venice", "the-merry-wives-of-windsor", "a-midsummer-nights-dream", \
            "much-ado-about-nothing", "pericles", "the-taming-of-the-shrew", "the-tempest", \
            "troilus-and-cressida", "twelfth-night", "the-two-gentlemen-of-verona", \
            "the-two-noble-kinsmen", "the-winters-tale"]

In [52]:
histories = ["henry-iv-part-1", "henry-iv-part-2", "henry-v", "henry-vi-part-1", "henry-vi-part-2", \
             "henry-vi-part-3", "henry-viii", "king-john", "richard-ii", "richard-iii"]

In [53]:
tragedies = ["antony-and-cleopatra", "coriolanus", "hamlet", "julius-caesar", "king-lear", \
             "macbeth", "othello", "romeo-and-juliet", "timon-of-athens", "titus-andronicus"]

In [69]:
new_cols = [key + '_mean' for key in char_df.columns]
new_cols.extend([key + '_median' for key in char_df.columns])
new_cols.extend([key + '_std' for key in char_df.columns])

type_df = pd.DataFrame(index = set(line_df.index.get_level_values(0)), columns = new_cols)

for play, name in index_without_number:
    play_slice = char_df.loc[(play, colon, colon)]

    means = play_slice.mean().add_suffix('_mean')
    medians =   play_slice.median().add_suffix('_median')
    stds = play_slice.std(ddof = 0).add_suffix('_std')
    
    new_row = pd.concat([means, medians, stds], axis = 0)

    type_df.loc[play] = new_row

In [71]:
type_df['play_type'] = -1

In [72]:
for i in type_df.index.tolist():
    if i in tragedies:
        type_df.loc[i, 'play_type'] = 2
    elif i in histories:
        type_df.loc[i, 'play_type'] = 1
    elif i in comedies:
        type_df.loc[i, 'play_type'] = 0

In [76]:
type_df.insert(0, 'play_type', type_df.pop('play_type'))

- Export data.

In [74]:
type_df.drop(columns = ['character_dies_mean', 'character_dies_median', 'character_dies_std'], inplace = True)

In [40]:
new_char_df.drop(columns = ['character_dies_median', 'character_dies_std'], inplace = True)

In [41]:
new_char_df.rename(columns = {'character_dies_mean': 'character_dies'}, inplace = True)

In [42]:
line_df.to_csv('../data/csv/ShakespeareCharacterLines_engineered.csv', index_label = ['play', 'name', 'line_number'])

In [43]:
new_char_df.to_csv('../data/csv/ShakespeareCharacterLines_character_corpus.csv', index_label = ['play', 'name'])

In [78]:
type_df.to_csv('../data/csv/ShakespeareCharacterLines_play_corpus.csv', index_label = ['play'])

- On to Step 4 ->